In [1]:
import sys
sys.path.append(r"../../utils")

import pandas as pd
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, precision_recall_curve, roc_curve, auc, average_precision_score

from questionnaires_aggregation import c_ssrs_aggregation
from utils import impute_from_column, simple_eda
from questions_columns import c_ssrs, sci_af_ca
from xgboost import XGBClassifier, XGBRegressor

In [2]:
# Splitting the data into train, validation, and test sets
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
c_ssrs = c_ssrs[:-3]
target_col = 'suicidal_behavior_time2'

sci_cols = ['is_sci_af_ca_missing', 'sci_af_ca_Factor1', 'sci_af_ca_Factor2',
             'sci_af_ca_Factor3', 'sci_af_ca_Factor4', 
             'sci_af_ca_Factor5'] + sci_af_ca

In [3]:
df = pd.read_csv(r"../create_dataset/data_for_prediction_task - measurement time axis.csv")
df_intake = df[df.measurement == 'time1']
df_target = df[df.measurement == 'time2']

In [4]:
df_target = df_target[[target_col, 'id']]
df_intake = df_intake.dropna(subset=sci_af_ca, thresh=20)
df_intake = df_intake[sci_cols + ['id']]
data_for_prediction = pd.merge(df_intake, df_target, on='id', how='inner')


In [5]:
label = data_for_prediction[target_col]
features = data_for_prediction[sci_cols]

In [6]:

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=test_ratio, random_state=42, stratify=label)

# Splitting the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), random_state=42, stratify=y_train)

In [7]:
# Create a CatBoostClassifier object with early stopping based on the validation set
eval_set  = [(X_val, y_val)]
model = CatBoostClassifier(random_state=42, class_weights = {0: 1, 1: 3}, eval_metric='AUC', reg_lambda = 1)

# Fit the model on the training set
model.fit(X_train, y_train, eval_set=eval_set, early_stopping_rounds = 10, verbose=0)#, use_best_model=True)

# Evaluate the model on the training and validation sets

# Make predictions on the training and validation sets
y_train_pred = model.predict(X_train)
y_train_prob = model.predict_proba(X_train)[:, 1]
y_val_pred = model.predict(X_val)
y_val_prob = model.predict_proba(X_val)[:, 1]


y_test_pred = model.predict(X_test)
y_test_prob = model.predict_proba(X_test)[:, 1]


In [8]:
# Function to calculate and print various performance metrics

def print_performance_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)
    average_precision = average_precision_score(y_true, y_prob)
    
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'PR AUC: {average_precision:.4f}')


In [9]:
# Print the performance metrics for the training set
print('Training set performance:')
print_performance_metrics(y_train, y_train_pred, y_train_prob)



Training set performance:
Accuracy: 0.9196
Precision: 0.7714
Recall: 0.9643
ROC AUC: 0.9906
PR AUC: 0.9774


In [10]:
# Print the performance metrics for the validation set
print('Validation set performance:')
print_performance_metrics(y_val, y_val_pred, y_val_prob)


Validation set performance:
Accuracy: 0.7368
Precision: 0.4545
Recall: 0.5556
ROC AUC: 0.7165
PR AUC: 0.4100


In [11]:
# Print the performance metrics for the validation set
print('Test set performance:')
print_performance_metrics(y_test, y_test_pred, y_test_prob)


Test set performance:
Accuracy: 0.6053
Precision: 0.3125
Recall: 0.5556
ROC AUC: 0.7011
PR AUC: 0.3815


In [12]:

# Get feature importance scores
feature_importance = model.get_feature_importance()
feature_names = X_train.columns.tolist()
# Print feature importance scores
print('Feature importance scores:')
for feature_name, score in sorted(zip(feature_names, feature_importance), reverse=1, key = lambda x: x[1]):
    print(f'{feature_name}: {score:.4f}')

Feature importance scores:
sci_af_ca_18: 11.0643
sci_af_ca_25: 7.6883
sci_af_ca_9: 7.0489
sci_af_ca_39: 6.1358
sci_af_ca_1: 4.5251
sci_af_ca_30: 4.4984
sci_af_ca_38: 4.2940
sci_af_ca_17: 3.5990
sci_af_ca_Factor1: 3.4610
sci_af_ca_Factor5: 3.1778
sci_af_ca_31: 3.0765
sci_af_ca_Factor3: 3.0499
sci_af_ca_11: 2.7533
sci_af_ca_15: 2.6016
sci_af_ca_24: 2.4741
sci_af_ca_34: 2.4495
sci_af_ca_20: 2.2697
sci_af_ca_27: 2.2389
sci_af_ca_5: 2.2162
sci_af_ca_35: 2.2134
sci_af_ca_12: 2.1679
sci_af_ca_7: 1.9195
sci_af_ca_23: 1.7997
sci_af_ca_13: 1.7670
sci_af_ca_21: 1.6498
sci_af_ca_14: 1.4476
sci_af_ca_29: 1.3070
sci_af_ca_22: 1.2232
sci_af_ca_8: 1.1910
sci_af_ca_32: 1.0346
sci_af_ca_33: 0.8347
sci_af_ca_Factor2: 0.7925
sci_af_ca_10: 0.5754
sci_af_ca_2: 0.4843
sci_af_ca_36: 0.4807
sci_af_ca_6: 0.2702
sci_af_ca_37: 0.2195
is_sci_af_ca_missing: 0.0000
sci_af_ca_Factor4: 0.0000
sci_af_ca_3: 0.0000
sci_af_ca_4: 0.0000
sci_af_ca_16: 0.0000
sci_af_ca_19: 0.0000
sci_af_ca_26: 0.0000
sci_af_ca_28: 0.0000
sci

cool, 
use the train-validation-test variables that you created in the last snippet, 
and train CatBoostClassifier, with early_stopping parameter based on the validation set.

after that look at the performance scores on the training & validation sets.
and the feature-importance scores

Since it's a complicated task, we need to use the following metrics: accuracy, precision, recall, precision-recall-auc, roc_auc